In [ ]:
from IPython.display import clear_output

In [ ]:
import os

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import train, tune
from ray.tune.schedulers import ASHAScheduler

In [ ]:
curr_path = os.path.abspath(os.getcwd())
data_path = os.path.join(curr_path, "plates")
data_path

In [ ]:
import shutil

if not os.path.isdir(data_path):
    !unzip {curr_path}/platesv2.zip -d {curr_path}/platesv2/
    !unzip {curr_path}/platesv2/plates.zip -d {curr_path}/tmp_plates/
    !mv {curr_path}/tmp_plates/plates/ {curr_path}/plates/
    !find {curr_path}/plates -name '.DS_Store' -type f -delete
    !rm -r {curr_path}/platesv2/
    !rm -r {curr_path}/tmp_plates
clear_output()

In [ ]:
import os, random

def create_val_dataset(val_size=0.3):
    random.seed("happy :)")

    train_path = os.path.join(data_path, "train")
    val_path = os.path.join(data_path, "val")
    try:
        os.mkdir(val_path)
    except FileExistsError:
        print("val already exist")
        return
    
    classes_name = os.listdir(train_path)
    for class_name in classes_name:
        class_path = os.path.join(train_path, class_name)
        images_filename = os.listdir(class_path)
        sample_size = int(val_size * len(images_filename))
        selected_val_images = random.sample(images_filename, sample_size)
        
        new_class_path = os.path.join(val_path, class_name)
        os.mkdir(new_class_path)
        [shutil.move(os.path.join(class_path, image_filename),new_class_path) for image_filename in selected_val_images]

create_val_dataset()

In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt


# def extract_plate_with_grabcut(image_path):
#     image = cv2.imread(image_path)
#     mask = np.zeros(image.shape[:2], np.uint8)
#     bgd_model = np.zeros((1, 65), np.float64)
#     fgd_model = np.zeros((1, 65), np.float64)
#     height, width = image.shape[:2]
#     rect = (int(width*0.1), int(height*0.1), int(width*0.8), int(height*0.8))
#     cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)
#     mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
#     result = image * mask2[:, :, np.newaxis]
#     contours, _ = cv2.findContours(mask2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     if contours:
#         largest_contour = max(contours, key=cv2.contourArea)
#         x, y, w, h = cv2.boundingRect(largest_contour)
#         cropped_result = result[y:y+h, x:x+w]
#     else:
#         cropped_result = result
#     return cropped_result

# def extract_class_bg(class_path):
#     imgs_filename = os.listdir(class_path)
#     for img_filename in imgs_filename:

#         # remove image background
#         img_path = os.path.join(class_path, img_filename)
#         print(img_path)
#         extracted_plate = extract_plate_with_grabcut(img_path)

#         # save img into new dataset path
#         cv2.imwrite(img_path, extracted_plate)

# def remove_datasets_background():
#     datasets_names = os.listdir(data_path)

#     for dataset_name in datasets_names:

#         dataset_path = os.path.join(data_path, dataset_name)
#         if dataset_name == "test":
#             extract_class_bg(dataset_path)
#         else:
#             classes_names = os.listdir(dataset_path)

#             for class_name in classes_names:

#                 # get class path
#                 class_path = os.path.join(dataset_path, class_name)

#                 extract_class_bg(class_path)
                
# remove_datasets_background()


In [ ]:
def get_transfer_learning_model(classifier_dropout, image_net):
    model = models.resnet152(weights=image_net)

    for param in model.parameters():
        param.requires_grad = True

    model.fc = nn.Sequential(
        nn.Dropout(classifier_dropout, inplace=True),
        nn.Linear(model.fc.in_features, 2),
    )

    # for layer in model.classifier:
    #     if isinstance(layer, nn.Dropout):
    #         layer.p = classifier_dropout
    
    return model

In [ ]:
import torch.utils
from torch.utils.data import DataLoader, Dataset

class AugmentedDataset(Dataset):
    def __init__(self, dataset, n_batch_size, transform=None):
        self.dataset = dataset
        self.n_batch_size = n_batch_size
        self.transform = transform

    def __len__(self):
        # Return the number of batches times the batch size
        return len(self.dataset) * (self.n_batch_size // len(self.dataset))

    def __getitem__(self, idx):
        # Get the original image and label
        img, label = self.dataset[idx % len(self.dataset)]
        
        # Apply transformations
        if self.transform:
            img = self.transform(img)
        
        return img, label
    
target_transforms = transforms.Compose([
    lambda x:torch.tensor(x), # or just torch.tensor
    lambda x:F.one_hot(x,2),
])

data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomRotation(degrees=(0, 360)),
        # transforms.RandomResizedCrop(256, scale=(0.5, 1), interpolation=transforms.InterpolationMode.BILINEAR),
        # transforms.AutoAugment(policy=transforms.autoaugment.AutoAugmentPolicy.IMAGENET),
        # transforms.RandomHorizontalFlip(),
        # transforms.RandomVerticalFlip(),
        # transforms.ColorJitter(brightness=(0.3, 1)),
        # # transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0)),
        # transforms.RandomEqualize(),
        # transforms.RandomGrayscale(p=0.2),
        # transforms.CenterCrop(224),
        # transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
        transforms.AutoAugment(policy=transforms.autoaugment.AutoAugmentPolicy.IMAGENET),
        transforms.RandomResizedCrop(232, scale=(0.5, 1), interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.ColorJitter(hue=(-0.5,0.5)),
        transforms.RandomEqualize(),
        # transforms.RandomGrayscale(p=0.2),
        #transforms.RandomGrayscale(p=1),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        # transforms.RandomRotation(degrees=(0, 360)),
        # transforms.RandomResizedCrop(256, scale=(0.8, 1), interpolation=transforms.InterpolationMode.BILINEAR),
        # transforms.AutoAugment(policy=transforms.autoaugment.AutoAugmentPolicy.IMAGENET),
        # transforms.RandomHorizontalFlip(),
        # transforms.RandomVerticalFlip(),
        # transforms.ColorJitter(brightness=(0.3, 1)),
        # # transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0)),
        # transforms.RandomEqualize(),
        # transforms.RandomGrayscale(p=0.2),
        # transforms.CenterCrop(224),
        # transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.RandomPerspective(distortion_scale=0.1, p=0.8, interpolation=3, fill=255),
        transforms.AutoAugment(policy=transforms.autoaugment.AutoAugmentPolicy.IMAGENET),
        transforms.RandomResizedCrop(232, scale=(0.05, 1), interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.ColorJitter(hue=(-0.5,0.5)),
        transforms.RandomEqualize(),
        # transforms.RandomGrayscale(p=0.2),
        #transforms.RandomGrayscale(p=1),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 2.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(232, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


data_dir = data_path
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),target_transform=target_transforms)
                  for x in ['train', 'val']}


augmented_dataset = {
    x: AugmentedDataset(image_datasets[x], 50, transform=data_transforms[x])
    for x in ['train', 'val']
}


class_names = image_datasets['train'].classes

dataloaders = {x: DataLoader(augmented_dataset[x], batch_size=50,
                                             shuffle=True, num_workers=10)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class_names, device

In [ ]:
from torchvision.utils import make_grid
from matplotlib import pyplot as plt

def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(10,10))
    plt.imshow(inp,)
    #if title is not None:
    #    plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

phases = ['train','val']
for phase in phases:
    print(phase)
    for inputs, labels in dataloaders[phase]:

        # Get a batch of training data
        inputs, classes = next(iter(dataloaders[phase]))
        classes = classes.max(axis=1)[1]
        
        # Make a grid from batch
        out = make_grid(inputs)

        imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def train_func(model, optimizer, exp_lr_scheduler, clip_value):
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    running_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(dataloaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target.float())

        total += output.size(0)
        running_loss += loss.item() * output.size(0)

        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()
        # accuracy
        _, predicted = torch.max(output.data, 1)
        _, correct_class = torch.max(target.data, 1)
        
        correct += (predicted == correct_class).sum().item()
    
    exp_lr_scheduler.step()
    
    return {
        "mean_loss": running_loss / total,
        "mean_accuracy": correct / total,
    }

def test_func(model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    running_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloaders['val']):
            
            data, target = data.to(device), target.to(device)
            outputs = model(data)

            # accuracy
            _, predicted = torch.max(outputs.data, 1)
            _, correct_class = torch.max(target.data, 1)
            total += target.size(0)
            correct += (predicted == correct_class).sum().item()

            # loss
            running_loss += F.cross_entropy(outputs, target.float()).item() * outputs.size(0)
    
    return {
        "mean_loss": running_loss / total,
        "mean_accuracy": correct / total,
    }

In [ ]:
import os
import tempfile

from ray.train import Checkpoint

def train_dishs(config, max_epochs=30, tunning=True):
    
    # Data Setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = get_transfer_learning_model(config['classifier_dropout'], config['image_net'])
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"], weight_decay=config['weight_decay'])
    
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=config['lr_scheduler_gamma'])
    for i in range(max_epochs):
        train_log = train_func(model, optimizer, exp_lr_scheduler, config['clip_value'])
        val_log = test_func(model)

        if tunning:
            with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
                checkpoint = None
                if (i + 1) % max_epochs == 0 and (val_log["mean_loss"] < 0.4):
                    # This saves the model to the trial directory
                    torch.save(
                        model.state_dict(),
                        os.path.join(temp_checkpoint_dir, "model.pth")
                    )
                    checkpoint = Checkpoint.from_directory(temp_checkpoint_dir)

                # Send the current training result back to Tune
                train.report(
                    {
                        "train_mean_loss": train_log["mean_loss"],
                        "train_mean_accuracy": train_log["mean_accuracy"],
                        "val_mean_loss": val_log["mean_loss"],
                        "val_mean_accuracy": val_log["mean_accuracy"],
                    },
                    checkpoint=checkpoint
                )
        else:
            print("-"*10, f"epoch: {i+1}/{max_epochs}","-"*10)
            print(f"train: {train_log}\nval: {val_log}")
    if not tunning:
        return {
            "model": model,
            "log": {
                "train": train_log,
                "val": val_log,
            },
        }

In [ ]:
import json

try:
    with open("best_result.json", "r") as f:
        curr_best_params = [json.load(f)]
except:
    curr_best_params = None

curr_best_params

In [ ]:
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

space = {
    "lr": hp.loguniform("lr", -3, -1),
    "momentum": hp.uniform("momentum", 0.1, 0.9),
    "classifier_dropout": hp.uniform("classifier_dropout", 0.5, 0.95),
    "weight_decay": hp.loguniform("weight_decay", -6, -2),
    "clip_value": hp.randint("clip_value", 1, 5+1),
    "lr_scheduler_gamma": hp.uniform("lr_scheduler_gamma", 0.5, 1.0),
    "image_net": hp.choice("image_net", ["IMAGENET1K_V1", "IMAGENET1K_V2"]),
}

metric = "val_mean_accuracy"
mode = "max"

hyperopt_search = HyperOptSearch(
    space,
    metric=metric,
    mode=mode,
    points_to_evaluate = curr_best_params
)

asas_scheduler = ASHAScheduler(
    time_attr='training_iteration',
    metric=metric,
    mode=mode,
    max_t=10,
    grace_period=1,
    reduction_factor=3,
    brackets=2
)

trainable_with_resources = tune.with_resources(train_dishs, {"cpu": 8, "gpu": 1})

tuner = tune.Tuner(
    trainable_with_resources,
    tune_config=tune.TuneConfig(
        num_samples=40,
        search_alg=hyperopt_search,
        scheduler=asas_scheduler
    ),
)
results = tuner.fit()

In [ ]:
best_result = results.get_best_result("val_mean_accuracy", mode="max")
best_result.metrics

In [ ]:
import json

with open("best_result.json", 'w') as f:
    json.dump(best_result.config, f, default=str)

In [ ]:
# best_result = results.get_best_result("val_mean_loss", mode="min")
# with best_result.checkpoint.as_directory() as checkpoint_dir:
#     state_dict = torch.load(os.path.join(checkpoint_dir, "model.pth"))

# model = get_transfer_learning_model(best_result.config['classifier_dropout']).to(device)
# model.load_state_dict(state_dict)
# model.classifier
# to_csv(model)

In [ ]:
from PIL import Image
import pandas as pd

def to_csv(model, batch_size=10):
    model.eval()
    PATH_TEST = os.path.join(data_path, "test")
    test_file_names = os.listdir(PATH_TEST)
    test_file_names.sort()

    submission_csv = {
        "id": [],
        "label": []
    }

    for file_name in test_file_names:
        id = file_name.split(".")[0]
        test_input = Image.open(os.path.join(PATH_TEST, file_name))
        test_input = data_transforms['test'](test_input).to(device).unsqueeze(0)
        with torch.no_grad():
            pred_test_label = model(test_input).max(1).indices.item()
            pred_test_label = class_names[pred_test_label]
        submission_csv['id'].append(id)
        submission_csv['label'].append(pred_test_label)

    submission_csv = pd.DataFrame(submission_csv).set_index("id")
    submission_csv.to_csv("submission.csv")

In [ ]:
import json

with open("best_result.json", "r") as f:
    best_config_loaded = json.load(f)

best_config_loaded

In [ ]:
best_config_train_model = train_dishs(best_config_loaded, max_epochs=10, tunning=False)

In [ ]:
new_model = best_config_train_model['model']

In [ ]:
# phases = ['train','val']
# for phase in phases:
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
#     for inputs, labels in dataloaders[phase]:
#         inputs, labels = inputs.to(device), labels.to(device)
        
#         #calculate val loss
#         outputs = new_model(inputs)
#         loss = F.cross_entropy(outputs, labels.float()).item()
#         print(loss)

#         # Get a batch of training data
#         classes = labels.max(axis=1)[1]
#         pred_classes = outputs.max(axis=1)[1]
#         print([class_names[i] for i in classes])
#         print(pred_classes == classes)
        
#         # Make a grid from batch
#         out = make_grid(inputs)

#         imshow(out.cpu(), title=[class_names[x] for x in classes])

In [ ]:
to_csv(new_model)

In [ ]:
df = results.get_dataframe()
df.head()

In [ ]:
# Analyze the results
import seaborn as sns
import matplotlib.pyplot as plt

log_plots = ["lr", "weight_decay"]
def plot_scatter(data, x_param, y_param="val_mean_loss"):
    
    if x_param == "clip_value":
        data[f"config/{x_param}"] = np.array(data[f"config/{x_param}"], dtype=int)
    sns.scatterplot(data=data, x=f'config/{x_param}', y=y_param)
    plt.title(f'{x_param} vs. {y_param}')
    plt.ylim(0,1.0)
    #plt.yscale("log")
    if x_param in log_plots:
        plt.xscale("log")
    plt.show()

# Create scatter plots for each hyperparameter
plot_scatter(df, 'lr')
plot_scatter(df, 'clip_value')
plot_scatter(df, 'momentum')
plot_scatter(df, 'classifier_dropout')
plot_scatter(df, 'weight_decay')
plot_scatter(df, 'lr_scheduler_gamma')
plot_scatter(df, 'image_net')

In [ ]:
!rm -r ./plates